# Exercises

This is the notebook containing the exercises for Feature Store, Model Monitor, and Clarify. Tested for these exercises was performed using __2 vCPU + 4 GiB notebook instance with Python 3 (TensorFlow 2.1 Python 3.6 CPU Optimized) kernel__.

## Staging

We'll begin by initializing some variables. These are often assumed to be present in code samples you'll find in the AWS documenation.

In [2]:
!pip install boto --upgrade
!pip install boto3 --upgrade
!pip install awscli --upgrade
!pip install -U sagemaker

  Using cached boto-2.49.0-py2.py3-none-any.whl (1.4 MB)
You should consider upgrading via the '/usr/local/bin/python3.6 -m pip install --upgrade pip' command.
  Using cached boto3-1.23.10-py3-none-any.whl (132 kB)
  Using cached botocore-1.26.10-py3-none-any.whl (8.8 MB)
  Using cached s3transfer-0.5.2-py3-none-any.whl (79 kB)
  Attempting uninstall: botocore
    Found existing installation: botocore 1.20.23
    Uninstalling botocore-1.20.23:
      Successfully uninstalled botocore-1.20.23
  Attempting uninstall: s3transfer
    Found existing installation: s3transfer 0.3.4
    Uninstalling s3transfer-0.3.4:
      Successfully uninstalled s3transfer-0.3.4
  Attempting uninstall: boto3
    Found existing installation: boto3 1.17.23
    Uninstalling boto3-1.17.23:
      Successfully uninstalled boto3-1.17.23
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 

In [3]:
import sagemaker
from sagemaker.session import Session
from sagemaker import get_execution_role

role = get_execution_role()
session = sagemaker.Session()
region = session.boto_region_name
bucket = session.default_bucket()

In [4]:
print(bucket)

sagemaker-us-east-1-560402809124


## Feature Store
---

Feature store is a special database to give ML systems a consistent data flow across training and inference workloads. It can ingest data in batches (for training) as well as serve input features to models with very low latency for real-time prediction.

For this exercise we'll work with a wine quality dataset: https://archive.ics.uci.edu/ml/datasets/wine+quality/

```P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis.
Modeling wine preferences by data mining from physicochemical properties. In Decision Support Systems, Elsevier, 47(4):547-553, 2009.```

In [5]:
import pandas as pd
from sklearn import datasets
import time
import uuid

In [6]:
data = datasets.load_wine()
df = pd.DataFrame(data['data'])
df.columns = data['feature_names']

If we leave the column names as-is, Feature Store won't be able to handle the `/` in `od280/od315_of_diluted_wines` (`/` is a delimiter Feature Store uses to manage how features are organized.)

In [7]:
df.rename(columns = {'od280/od315_of_diluted_wines':'od280_od315_of_diluted_wines'}, inplace=True)

Once we have our data, we can create a feature group. Remember to attach event time and ID columns - Feature Store needs them.

In [8]:
# Add a feature group
df["EventTime"] = time.time()
df["id"] = range(len(df))

# TODO: Create feature group
from sagemaker.feature_store.feature_group import FeatureGroup

feature_group = FeatureGroup(
    name='wine-features', sagemaker_session=session)

# TODO: Load Feature definitions

feature_group.load_feature_definitions(data_frame=df)


[FeatureDefinition(feature_name='alcohol', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='malic_acid', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='ash', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='alcalinity_of_ash', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='magnesium', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='total_phenols', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='flavanoids', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='nonflavanoid_phenols', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='proanthocyanins', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='color_intensity',

The feature group is not created until we call the `create` method, let's do that now:

In [9]:
# Create the feature store:

feature_group.create(
    s3_uri=f"s3://{bucket}/features",
    record_identifier_name="id",
    event_time_feature_name="EventTime",
    role_arn=role,
    enable_online_store=True,
)



ResourceInUse: An error occurred (ResourceInUse) when calling the CreateFeatureGroup operation: Resource Already Exists: FeatureGroup with name wine-features already exists. Choose a different name.

Lastly, ingest some data into your feature group:

In [18]:
runtime = session.boto_session.client(
  'sagemaker-featurestore-runtime',
  region_name=region)

runtime.get_record(
    FeatureGroupName="wine-features",
    RecordIdentifierValueAsString="0")

{'ResponseMetadata': {'RequestId': '5aebc06d-da7e-4305-8564-b0e8f10570c6',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '5aebc06d-da7e-4305-8564-b0e8f10570c6',
   'content-type': 'application/json',
   'content-length': '823',
   'date': 'Wed, 11 Jan 2023 13:46:44 GMT'},
  'RetryAttempts': 0},
 'Record': [{'FeatureName': 'alcohol', 'ValueAsString': '14.23'},
  {'FeatureName': 'malic_acid', 'ValueAsString': '1.71'},
  {'FeatureName': 'ash', 'ValueAsString': '2.43'},
  {'FeatureName': 'alcalinity_of_ash', 'ValueAsString': '15.6'},
  {'FeatureName': 'magnesium', 'ValueAsString': '127.0'},
  {'FeatureName': 'total_phenols', 'ValueAsString': '2.8'},
  {'FeatureName': 'flavanoids', 'ValueAsString': '3.06'},
  {'FeatureName': 'nonflavanoid_phenols', 'ValueAsString': '0.28'},
  {'FeatureName': 'proanthocyanins', 'ValueAsString': '2.29'},
  {'FeatureName': 'color_intensity', 'ValueAsString': '5.64'},
  {'FeatureName': 'hue', 'ValueAsString': '1.04'},
  {'FeatureName': 'od280_o

In [19]:
# TODO

feature_group.ingest(data_frame=df, max_workers=3, wait=True)

Failed to ingest row 120: An error occurred (ValidationError) when calling the PutRecord operation: Validation Error: Provided values for Feature(s) [TARGET] which do not exist in FeatureGroup [wine-features].
Failed to ingest row 0: An error occurred (ValidationError) when calling the PutRecord operation: Validation Error: Provided values for Feature(s) [TARGET] which do not exist in FeatureGroup [wine-features].
Failed to ingest row 60: An error occurred (ValidationError) when calling the PutRecord operation: Validation Error: Provided values for Feature(s) [TARGET] which do not exist in FeatureGroup [wine-features].
Failed to ingest row 1: An error occurred (ValidationError) when calling the PutRecord operation: Validation Error: Provided values for Feature(s) [TARGET] which do not exist in FeatureGroup [wine-features].
Failed to ingest row 121: An error occurred (ValidationError) when calling the PutRecord operation: Validation Error: Provided values for Feature(s) [TARGET] which d

IngestionError: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177] -> Failed to ingest some data into FeatureGroup wine-features

Great job! You've demonstrated your understanding of creating feature groups and ingesting data into them using Feature Store. Next up we'll cover Model Monitor!

## Model Monitor

In this exercise we'll create a monitoring schedule for a deployed model. We're going to provide code to help you deploy a model and get started, so that you can focus on Model Monitor for this exercise. __Remember to clean up your model before you end a work session__. We'll provide some code at the end to help you clean up your model. We'll begin by reloading our data from the previous exercise.



In [20]:
data = datasets.load_wine()
df = pd.DataFrame(data['data'])
df.columns = data['feature_names']
df.rename(columns = {'od280/od315_of_diluted_wines':'od280_od315_of_diluted_wines'}, inplace=True)

We also need to put the target variable in the first column per the docs for our chosen algorithm: https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html

In [21]:
df["TARGET"] = data['target']
df.set_index(df.pop('TARGET'), inplace=True)
df.reset_index(inplace=True)

Now we'll upload the data to S3 as train and validation data:

In [22]:
delimiter = int(len(df)/2)
train, test = df.iloc[delimiter:], df.iloc[:delimiter]

train.to_csv("train.csv", header=False, index=False)
test.to_csv("validation.csv", header=False, index=False)

val_location = session.upload_data('./validation.csv', key_prefix="data")
train_location = session.upload_data('./train.csv', key_prefix="data")

s3_input_train = sagemaker.inputs.TrainingInput(s3_data=train_location, content_type='csv')
s3_input_validation = sagemaker.inputs.TrainingInput(s3_data=val_location, content_type='csv')

In [23]:
algo_image = sagemaker.image_uris.retrieve("xgboost", region, version='latest')
s3_output_location = f"s3://{bucket}/models/wine_model"

model=sagemaker.estimator.Estimator(
    image_uri=algo_image,
    role=role,
    instance_count=1,
    instance_type='ml.m4.xlarge',
    volume_size=5,
    output_path=s3_output_location,
    sagemaker_session=sagemaker.Session()
)

model.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        objective='reg:linear',
                        early_stopping_rounds=10,
                        num_round=200)


model.fit({'train': s3_input_train, 'validation': s3_input_validation})

2023-01-11 13:48:33 Starting - Starting the training job...ProfilerReport-1673444912: InProgress
...
2023-01-11 13:49:30 Starting - Preparing the instances for training.........
2023-01-11 13:50:54 Downloading - Downloading input data......
2023-01-11 13:51:49 Training - Downloading the training image...
2023-01-11 13:52:35 Uploading - Uploading generated training model.Arguments: train
[2023-01-11:13:52:29:INFO] Running standalone xgboost training.
[2023-01-11:13:52:29:INFO] File size need to be processed in the node: 0.01mb. Available memory size in the node: 8622.09mb
[2023-01-11:13:52:29:INFO] Determined delimiter of CSV input is ','
[13:52:29] S3DistributionType set as FullyReplicated
[13:52:29] 89x13 matrix with 1157 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2023-01-11:13:52:29:INFO] Determined delimiter of CSV input is ','
[13:52:29] S3DistributionType set as FullyReplicated
[13:52:29] 89x13 matrix with 1157 entries loaded from /opt/ml/i

Now that your training job has finished, you can perform the first task in this exercise: creating a data capture config. Configure your model to sample `34%` of inferences:

In [24]:
# TODO
from sagemaker.model_monitor import DataCaptureConfig

capture_uri = 's3://{bucket}/data-capture'

data_capture_config = DataCaptureConfig(
    enable_capture=True,
    sampling_percentage=34,
    destination_s3_uri=capture_uri
)

Great! We'll use your config to deploy a model below:

In [25]:
xgb_predictor = model.deploy(
    initial_instance_count=1, instance_type='ml.m4.xlarge',
    data_capture_config=data_capture_config
)

-------!

Great! You should see an indicator like this when the deployment finishes:

```
-----------------!
```
We can test your deployment like so:

In [19]:
xgb_predictor.serializer = sagemaker.serializers.CSVSerializer()
inputs = test.copy()
# Drop the target variable
inputs = inputs.drop(columns=inputs.columns[0])
x_pred = xgb_predictor.predict(inputs.sample(5).values).decode('utf-8')
x_pred

'0.6030303239822388,0.6030303239822388,0.7861111164093018,0.6030303239822388,0.6030303239822388'

All systems go! To finish up the exercise, we're going to provide you with a DefaultModelMonitor and a suggested baseline. Combine the `xgb_predictor` and the provided `my_monitor` to configure the monitoring schedule for _hourly_ monitoring.

In [20]:
from sagemaker.model_monitor import DefaultModelMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat

my_monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    volume_size_in_gb=20,
    max_runtime_in_seconds=3600,
)

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: .
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


In [21]:
my_monitor.suggest_baseline(
    baseline_dataset=f's3://{bucket}/data/train.csv',
    dataset_format=DatasetFormat.csv(header=False),
)

INFO:sagemaker:Creating processing-job with name baseline-suggestion-job-2023-01-11-01-44-28-890



Job Name:  baseline-suggestion-job-2023-01-11-01-44-28-890
Inputs:  [{'InputName': 'baseline_dataset_input', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-560402809124/data/train.csv', 'LocalPath': '/opt/ml/processing/input/baseline_dataset_input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'monitoring_output', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-560402809124/model-monitor/baselining/baseline-suggestion-job-2023-01-11-01-44-28-890/results', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]
.........................................2023-01-11 01:51:10,964 - matplotlib.font_manager - INFO - Generating new fontManager, this may take some time...
2023-01-11 01:51:11.505486: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.s

Below, provide the monitoring schedule:

In [25]:
# TODO
from sagemaker.model_monitor import CronExpressionGenerator

my_monitor.create_monitoring_schedule(
    monitor_schedule_name='kevins-monitoring-schedule',
    endpoint_input=xgb_predictor.endpoint_name,
    statistics=my_monitor.baseline_statistics(),
    constraints=my_monitor.suggested_constraints(),
    schedule_cron_expression=CronExpressionGenerator.hourly()
)

INFO:sagemaker.model_monitor.model_monitoring:Creating Monitoring Schedule with name: kevins-monitoring-schedule


Great job! You can check that your schedule was created by selecting the `SageMaker components and registries` tab on the far left.

In this exercise you configured Model Monitor to watch a simple model. Next, we'll monitor the same deployment for explainability.

__REMINDER:__ Don't leave your model deployed overnight. If you aren't going to follow up with the Clarify exercise within a few hours, use the code below to remove your model:

In [26]:
monitors = xgb_predictor.list_monitors()
for monitor in monitors:
    monitor.delete_monitoring_schedule()

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: .
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.



Deleting Monitoring Schedule with name: kevins-monitoring-schedule


INFO:sagemaker.model_monitor.model_monitoring:Deleting Data Quality Job Definition with name: data-quality-job-definition-2023-01-11-01-55-52-756


In [27]:
xgb_predictor.delete_endpoint()

INFO:sagemaker:Deleting endpoint configuration with name: xgboost-2023-01-11-01-40-14-643
INFO:sagemaker:Deleting endpoint with name: xgboost-2023-01-11-01-40-14-643


## Clarify

For the last exercise we'll deploy an explainability monitor using Clarify. We're going to use the model that you deployed in the last exercise, but if you cleaned up your deployments from the previous exercise, that's ok! You can rerun the deployment from the previous exercise up to the point where we deployed our model. It'll look like this:

```python
xgb_predictor = model.deploy(
    initial_instance_count=1, instance_type='ml.m4.xlarge',
    data_capture_config=data_capture_config
)
```

Once your model is deployed, you can come back here. _REMINDER_: you need to clean up your deployment, don't leave it running overnight. We'll provide some code at the end to delete your deployment.

## Prep

We'll begin by reloading our data from the previous exercise.

In [26]:
data = datasets.load_wine()
df = pd.DataFrame(data['data'])
df.columns = data['feature_names']
df.rename(columns = {'od280/od315_of_diluted_wines':'od280_od315_of_diluted_wines'}, inplace=True)

We also need to put the target variable in the first column per the docs for our chosen algorithm: https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html

In [27]:
df["TARGET"] = data['target']
df.set_index(df.pop('TARGET'), inplace=True)
df.reset_index(inplace=True)

Now we'll upload the data to S3 as train and validation data:

In [28]:
delimiter = int(len(df)/2)
train, test = df.iloc[delimiter:], df.iloc[:delimiter]

train.to_csv("train.csv", header=False, index=False)
test.to_csv("validation.csv", header=False, index=False)

val_location = session.upload_data('./validation.csv', key_prefix="data")
train_location = session.upload_data('./train.csv', key_prefix="data")

s3_input_train = sagemaker.inputs.TrainingInput(s3_data=train_location, content_type='csv')
s3_input_validation = sagemaker.inputs.TrainingInput(s3_data=val_location, content_type='csv')

Great! Our data is staged and our model is deployed - let's monitor it for explainability. We need to define three config objects, the `SHAPConfig`, the `ModelConfig`, and the `ExplainabilityAnalysisConfig`. Below, we provide the `SHAPConfig`.

In [29]:
shap_config = sagemaker.clarify.SHAPConfig(
    baseline=[train.mean().astype(int).to_list()[1:]],
    num_samples=int(train.size),
    agg_method="mean_abs",
    save_local_shap_values=False,
)

Next up, fill in the blanks to define the `ModelConfig` and `ExplainabilityAnalysisConfig`.

In [30]:
# TODO
model_config = sagemaker.clarify.ModelConfig(
    model_name=xgb_predictor.endpoint_name,
    instance_count=1,
    instance_type='m1.m4.xlarge',
    content_type="text/csv",
    accept_type='text/csv',
)

analysis_config = sagemaker.model_monitor.ExplainabilityAnalysisConfig(
    explainability_config=shap_config,
    model_config=model_config,
    headers=train.columns.to_list()[1:],
    )

Before we apply our config, we need to create the monitor object. This is what we'll apply all our config to.

In [31]:
model_explainability_monitor = sagemaker.model_monitor.ModelExplainabilityMonitor(
    role=role,
    sagemaker_session=session,
    max_runtime_in_seconds=1800,
)

Everything's ready! Below, create a monitoring schedule using the configs we created. Set the schedule to run _daily_.

In [35]:
# TODO 
from sagemaker.model_monitor import CronExpressionGenerator


explainability_uri =f's3://{bucket}/model-explainability'
model_explainability_monitor.create_monitoring_schedule(
    output_s3_uri=explainability_uri,
    analysis_config=analysis_config,
    endpoint_input=xgb_predictor.endpoint_name,
    schedule_cron_expression=CronExpressionGenerator.daily()
)

Way to go! You can check that your schedule was created by selecting the `SageMaker components and registries` tab on the far left.

In this exercise you deployed a monitor for explainability to your SageMaker endpoint. This is the last exercise - you'll apply these learnings again in your Project at the end of the course.



__REMINDER:__ Don't leave your model deployed overnight. Use the code below to remove your model:

In [37]:
monitors = xgb_predictor.list_monitors()
for monitor in monitors:
    monitor.delete_monitoring_schedule()


Deleting Monitoring Schedule with name: monitoring-schedule-2023-01-11-14-07-35-661


In [38]:
xgb_predictor.delete_endpoint()